In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pathlib import Path
import h5py
import pickle
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from src.data.file_utils import GetTV
from src.models import model, predict

In [3]:
models = predict.list_models()
for idx, model in enumerate(models):
    print(idx, model)

0 ../models/weighted_outer_allold_set.pkl
1 ../models/lr_tv_inv_outer.pkl
2 ../models/weighted_outer_all.pkl
3 ../models/weighted_outer_allnew_half_set.pkl
4 ../models/lr_inversion_manual.pkl
5 ../models/weighted_outer_l-mode.pkl
6 ../models/lr.pkl
7 ../models/lr_supervised_mdl.pkl


# Data Prep

In [5]:
prep_filename = 'tv_inv_outer'
prediction_filename = 'lr'
algorithm = 'linear'
inversion_to_point_model_index = 1
split_ratio = 0.2

In [6]:
tv_path = Path('../data/raw/tv_images/l-mode')
label_path = Path('../data/processed/inversion_data')
prep_path = Path('../data/processed/hdf5')
model_path = Path('../models')

In [7]:
tv = GetTV(tv_path)
files = tv.list_files()
file_lengths = tv.file_lengths()
cumulative_lengths = np.insert(np.cumsum(file_lengths), 0, 0)
tv_dim = tv.load(files[0], 'vid').shape
inversion_dim = tv.load(files[0], 'inverted').shape

In [37]:
def file_split_compile_prep(split_ratio,
                      prep_path,
                      prep_filename,
                      files):
    # split by file
    train_files, test_files = train_test_split(files, test_size=split_ratio, random_state=30)
    [print(file.stem) for file in train_files]
    print('-------------------')
    [print(file.stem) for file in test_files]

    points_train = []
    tv_train = []
    inverted_train = []
    points_test = []
    tv_test = []
    inverted_test = []
    
    with open(models[inversion_to_point_model_index], 'rb') as f:
        inversion_model = pickle.load(f)

    for _, file in enumerate(train_files):
        frames = tv.load(file, 'frames').astype('int')
        tv_image = tv.load(file, 'vid')[frames]
        inversion = tv.load(file, 'inverted')
        inversion_vid2 = inversion.reshape((len(inversion), -1))
        print(inversion.shape)
        print(tv_image.shape)
        for i in range(len(frames)):
            tv_train.append(tv_image[i])
            inverted_train.append(inversion[i])
            points_train.append(inversion_model.predict(inversion_vid2[i].reshape(1, -1)))
            
    for _, file in enumerate(test_files):
        frames = tv.load(file, 'frames').astype('int')
        tv_image = tv.load(file, 'vid')[frames]
        inversion = tv.load(file, 'inverted')
        inversion_vid2 = inversion.reshape((len(inversion), -1))
        for i in range(len(frames)):
            tv_test.append(tv_image[i])
            inverted_test.append(inversion[i])
            points_test.append(inversion_model.predict(inversion_vid2[i].reshape(1, -1)))
            
    with h5py.File(prep_path / str(prep_filename + '.h5'), 'w') as f:
        f.create_dataset('vid_train', data=tv_train)
        f.create_dataset('inverted_train', data=inverted_train)
        f.create_dataset('points_train', data=points_train)
        f.create_dataset('vid_test', data=tv_test)
        f.create_dataset('inverted_test', data=inverted_test)
        f.create_dataset('points_test', data=points_test)
    print(str(prep_path / str(prep_filename + '.h5')),'has been saved!')

In [38]:
file_split_compile_prep(split_ratio, prep_path, prep_filename, files)

emission_structure_pu_cam240perp_190109
emission_structure_pu_cam240perp_190113
emission_structure_pu_cam240perp_190114
emission_structure_pu_cam240perp_190116
-------------------
emission_structure_pu_cam240perp_190115
emission_structure_pu_cam240perp_190110
(120, 201, 201)
(120, 240, 720)


/scratch/gpfs/nc1514/plasma-tv/myenv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


(121, 201, 201)
(121, 240, 720)
(121, 201, 201)
(121, 240, 720)
(120, 201, 201)
(120, 240, 720)
../data/processed/hdf5/tv_inv_outer.h5 has been saved!


# Train Test Deploy

### Train

In [14]:
flatten = lambda x : x.reshape(len(x), -1)

In [15]:
file_name = prep_filename + '.h5'

with h5py.File(prep_path / file_name, 'r') as f:
    vid_train = f['vid_train'][:]
    inverted_train = f['inverted_train'][:]
    points_train = f['points_train'][:]
    vid_test = f['vid_test'][:]
    inverted_test = f['inverted_test'][:]
    points_test = f['points_test'][:]

files = tv.list_files()
elevation = tv.load(files[0], 'elevation')[0]
radii = tv.load(files[0], 'radii')[0]
vid_shape = tv.load(files[0], 'vid')[0].shape

In [16]:
X_train = flatten(vid_train)
X_test = flatten(vid_test)
y_train = flatten(points_train)
y_test = flatten(points_test)

In [21]:
mdl = LinearRegression()
mdl.fit(X_train, y_train)

LinearRegression()

### Test

In [24]:
r_predict = mdl.predict(X_test)
real_predict = np.array([radii[np.round(r_predict[:,0]).astype(int)],elevation[np.round(r_predict[:,1]).astype(int)]]).T * 100 # from meter to cm (dimension is n x 2)
real_y = np.array([radii[np.round(y_test[:,0]).astype(int)],elevation[np.round(y_test[:,1]).astype(int)]]).T * 100
dist = np.linalg.norm(real_predict - real_y, axis=1)
z_err = np.abs(real_predict[:,1] - real_y[:,1])
print(f"RMS (cm) : {np.sqrt(np.square(dist).mean())}")
print(f"Z RMS (cm) : {np.sqrt(np.square(z_err).mean())}")

RMS (cm) : 1.125494360923767
Z error (cm) : 0.8020638227462769


### Deploy

In [14]:
pickle.dump(mdl, open(model_path / 'lr.pkl', 'wb'))
coef = mdl.coef_
intercept = mdl.intercept_
with open(model_path / 'lr_coef.txt', 'w') as f:
    for i in range(coef.shape[1]):
        f.write(str(coef[0][i]) + '\n')
    for i in range(coef.shape[1]):
        f.write(str(coef[1][i]) + '\n')

with open(model_path / 'lr_intercept.txt', 'w') as f:
    for i in range(intercept.shape[0]):
        f.write(str(intercept[i]) + '\n')

In [4]:
mdl_path = "../models/lr_coef.txt"  # Replace with your file path

with open(mdl_path, "r") as file:
    coef = file.readlines()

mdl_path = "../models/lr_intercept.txt"  # Replace with your file path

with open(mdl_path, "r") as file:
    inter = file.readlines()
    
coef_ = np.array([float(i) for i in coef])
inter_ = np.array([float(i) for i in inter])
r_coef_ = coef_[:len(coef_)//2]
z_coef_ = coef_[len(coef_)//2:]

In [5]:
# example index 1
pred_r = np.dot(X_test, r_coef_) + inter_[0] # left half
pred_z = np.dot(X_test, z_coef_) + inter_[1] # right half
plt.plot(pred_r, label='predict_r')
plt.plot(pred_z, label='predict_z')
plt.legend()
plt.show()

NameError: name 'X_test' is not defined

# Downsample Tests

In [23]:
# downsampling functions
downsample2d = lambda x, factor : x[:, ::factor, ::factor]
flatten = lambda x : x.reshape(len(x), -1)

def regr_scoring(X_train, X_test, y_train, y_test):
    regr = LinearRegression()
    regr.fit(X_train, y_train)
    r_predict = regr.predict(X_test)
    real_predict = np.array([radii[np.round(r_predict[:,0]).astype(int)],elevation[np.round(r_predict[:,1]).astype(int)]]).T * 100 # from meter to cm (dimension is n x 2)
    real_y = np.array([radii[np.round(y_test[:,0]).astype(int)],elevation[np.round(y_test[:,1]).astype(int)]]).T * 100
    dist = np.linalg.norm(real_predict - real_y, axis=1)
    z_err = np.abs(real_predict[:,1] - real_y[:,1])
    return r_predict, dist, z_err

In [19]:
downsamples = range(1,10)
numpixel_array = []
r_sq_array = []
rms_array = []
rms_z_err_array = []

for n in downsamples:
    print(n)
    vid_train_down = downsample2d(vid_train, n)
    vid_test_down = downsample2d(vid_test, n)
    
    vid_train_down_reshape = flatten(vid_train_down)
    vid_test_down_reshape = flatten(vid_test_down)
    points_train_reshape = flatten(points_train)
    points_test_reshape = flatten(points_test)
    
    numpixel = vid_train_down_reshape.shape[1]
    
    _, dist, z_err = regr_scoring(vid_train_down_reshape, vid_test_down_reshape, points_train_reshape, points_test_reshape)
    
    numpixel_array.append(numpixel)
    rms_array.append(np.sqrt(np.square(dist).mean()))
    rms_z_err_array.append(np.sqrt(np.square(z_err).mean()))

1


In [ ]:
plt.plot(downsamples, rms_z_err_array)
plt.xlabel('Downsample Factor')
plt.ylabel('RMS Z Error (cm)')
plt.title('RMS Z Error vs Downsample Factor')
plt.show()

In [ ]:
plt.plot(downsamples, rms_array)
plt.xlabel('Downsample Factor')
plt.ylabel('RMS Error (cm)')
plt.title('RMS Error vs Downsample Factor')
plt.show()

In [ ]:
print('Downsample factor', 'RMS Error (cm)')
for i in range(len(rms_array)):
    print(i, rms_array[i])

## Filter Bad Data

In [ ]:
down_n = 1

In [ ]:
vid_train_down = downsample2d(vid_train, down_n)
vid_test_down = downsample2d(vid_test, down_n)

vid_train_down_reshape = flatten(vid_train_down)
vid_test_down_reshape = flatten(vid_test_down)
points_train_reshape = flatten(points_train)
points_test_reshape = flatten(points_test)

numpixel = vid_train_down_reshape.shape[1]

r_predict, r_sq, dist, rms, z_err = regr_scoring(vid_train_down_reshape, vid_test_down_reshape, points_train_reshape, points_test_reshape)

print(r_sq)
print(f"RMS (cm) : {rms}")

In [ ]:
plt.plot(z_err, label='Z Error')
plt.plot(dist, label='Distance Error')
plt.legend()
plt.show()

In [ ]:
print(np.where(z_err > 10))

In [ ]:
z_err_filter = z_err[z_err < np.percentile(z_err, 100)]
dist_filter = dist[z_err < np.percentile(dist, 100)]
z_err_filter_rms = np.sqrt(np.square(z_err_filter).mean())
dist_filter_rms = np.sqrt(np.square(dist_filter).mean())
print(f"RMS Z Error (cm) : {z_err_filter_rms}")
print(f"RMS Distance Error (cm) : {dist_filter_rms}")

In [ ]:
plt.plot(z_err_filter, label='Z Error')
plt.plot(dist_filter, label='Distance Error')
plt.legend()
plt.show()